# `quimb` exploration

In [1]:
import quimb as q
import qutip


In [2]:
import pickle
import numpy as np

import interaction_constants
from qubit_system.geometry.regular_lattice_1d import RegularLattice1D
from qubit_system.qubit_system_classes import EvolvingQubitSystem
from qubit_system.utils.ghz_states import StandardGHZState
from qubit_system.utils.states import get_ground_states, get_exp_list

from qubit_system.utils.interpolation import get_hamiltonian_coeff_linear_interpolation, \
    get_hamiltonian_coeff_interpolation

In [10]:
N = 4

psi_0_qutip = qutip.tensor(get_ground_states(N))
psi_0 = q.quimbify(psi_0_qutip, sparse=True)

In [15]:
ghz_state_qutip = StandardGHZState(N).get_state_tensor(True)
ghz_state = q.quimbify(ghz_state_qutip, sparse=True)

In [28]:
exp_list = get_exp_list(N)

In [29]:
sx_list, sy_list, sz_list = exp_list

In [35]:
for i in range(N):
        print(qutip.tensor(sx_list[i])._data == q.ikron(q.pauli("X"), dims=[2] * N, inds=i))

[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  Tr

In [36]:
N_RYD = 50
C6 = interaction_constants.get_C6(N_RYD)

LATTICE_SPACING = 1.5e-6

print(f"C6: {C6:.3e}")
characteristic_V = C6 / (LATTICE_SPACING ** 6)
print(f"Characteristic V: {characteristic_V:.3e} Hz")


C6: 1.555e-26
Characteristic V: 1.365e+09 Hz


In [103]:
geometry = RegularLattice1D(LATTICE_SPACING)

def get_hamiltonian():
    sx_list, sy_list, sz_list = get_exp_list(N)
    time_independent_terms = 0
    Omega_coeff_terms = 0
    Delta_coeff_terms = 0

    for i in range(N):
        Omega_coeff_terms += 1 / 2 * sx_list[i]
        n_i = (sz_list[i] + qutip.qeye(1)) / 2
        Delta_coeff_terms -= n_i

        for j in range(i):
            n_j = (sz_list[j] + qutip.qeye(1)) / 2
            time_independent_terms += C6 / geometry.get_distance(i, j) ** 6 * n_i * n_j
    return [
        time_independent_terms,
        [Omega_coeff_terms, None],
        [Delta_coeff_terms, None]
    ]

hamiltonian_qutip = get_hamiltonian()

In [109]:
def get_hamiltonian_quimb():
    sx = q.pauli("X")
    qnum = (sz + q.identity(2)) / 2
    dims = [2] * N

    time_independent_terms = 0
    Omega_coeff_terms = 0
    Delta_coeff_terms = 0

    for i in range(N):
        Omega_coeff_terms += 1 / 2 * q.ikron(sx, dims=dims, inds=i)
        n_i = q.ikron(qnum, dims=dims, inds=i)
        Delta_coeff_terms -= n_i

        for j in range(i):
            n_j = q.ikron(qnum, dims=dims, inds=j)

            time_independent_terms += C6 / geometry.get_distance(i, j) ** 6 * n_i * n_j
    return [
        time_independent_terms,
        [Omega_coeff_terms, None],
        [Delta_coeff_terms, None]
    ]


hamiltonian_quimb = get_hamiltonian_quimb()

In [40]:
(qutip.sigmaz() + qutip.qeye(1)) / 2


Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [52]:
(sz_list[0] + qutip.qeye(1)) / 2

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

In [50]:
qutip.ket('e')
g = qutip.ket('e')
e = qutip.ket('g')

qg = q.qu(g)
qe = q.qu(e)

In [65]:
q.expec(q.num(2), q.qu(qutip.ket('g')))

0j

In [66]:
q.num(2)

[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

In [68]:
q.num(2).T

[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]

In [75]:
q.create(2) @ qe

[[0.+0.j]
 [1.+0.j]]

In [76]:
qe

[[1.+0.j]
 [0.+0.j]]

In [85]:
sz = q.pauli("Z")
qnum = (sz + q.identity(2)) / 2
print(qnum)

[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [108]:
dims = [2] * N

q.ikron(qnum, dims=dims, inds=0) / 2

[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0. 

In [117]:
np.all(hamiltonian_qutip[0].data == hamiltonian_quimb[0]) and \
np.all(hamiltonian_qutip[1][0].data == hamiltonian_quimb[1][0]) and \
np.all(hamiltonian_qutip[2][0].data == hamiltonian_quimb[2][0])



True

In [121]:
qg

[[0.+0.j]
 [1.+0.j]]

In [130]:
sx = q.pauli("X")
q.measure(qg, sx)


(1.0, [[0.707107+0.j]
  [0.707107+0.j]])

In [5]:
# import qubit_system.qubit_system_classes_qiumb as qscq
from qubit_system.qubit_system_classes_quimb import TimeIndependentEvolvingQubitSystem as TIEQS
t = 1
N = 4
e_qs = TIEQS(
        N=N, V=1, geometry=RegularLattice1D(),
        Omega=1, Delta=1,
        ghz_state=StandardGHZState(N)
)

In [53]:
evo = q.Evolution(e_qs.psi_0, e_qs.get_hamiltonian())

In [30]:
for i in evo.at_times([ 1]):
    print(i)

[[ 4.703167e-06+3.692010e-07j]
 [-8.233488e-06+1.089607e-04j]
 [-1.124813e-05+1.087117e-04j]
 [-2.276932e-03-1.453854e-04j]
 [-1.124813e-05+1.087117e-04j]
 [-2.270409e-03-2.168209e-04j]
 [-2.270458e-03-2.178574e-04j]
 [ 2.274377e-03-4.758548e-02j]
 [-8.233488e-06+1.089607e-04j]
 [-2.276822e-03-1.453983e-04j]
 [-2.270409e-03-2.168209e-04j]
 [ 2.275133e-03-4.758543e-02j]
 [-2.276932e-03-1.453854e-04j]
 [ 2.275133e-03-4.758543e-02j]
 [ 2.274377e-03-4.758548e-02j]
 [ 9.954346e-01-1.454966e-04j]]


In [17]:
evo.pt == evo._p0

[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]]

In [40]:
evo.pt


[[ 4.703167e-06+3.692010e-07j]
 [-8.233488e-06+1.089607e-04j]
 [-1.124813e-05+1.087117e-04j]
 [-2.276932e-03-1.453854e-04j]
 [-1.124813e-05+1.087117e-04j]
 [-2.270409e-03-2.168209e-04j]
 [-2.270458e-03-2.178574e-04j]
 [ 2.274377e-03-4.758548e-02j]
 [-8.233488e-06+1.089607e-04j]
 [-2.276822e-03-1.453983e-04j]
 [-2.270409e-03-2.168209e-04j]
 [ 2.275133e-03-4.758543e-02j]
 [-2.276932e-03-1.453854e-04j]
 [ 2.275133e-03-4.758543e-02j]
 [ 2.274377e-03-4.758548e-02j]
 [ 9.954346e-01-1.454966e-04j]]

In [39]:
evo.t


0.09568039050677621

In [37]:
evo.update_to(1)

In [43]:
qutip.basis(2, 1).data == q.ket([0, 1])

matrix([[ True],
        [ True]])

In [48]:
q.kron(q.ket([0, 1]), q.ket([0, 1], sparse=True)).argmax()

3

In [52]:
make_immutable(qe)

AttributeError: module 'quimb' has no attribute 'make_immutable'

In [54]:
for i in evo.at_times([0, 1]):
    print(i)
    
    

[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [1.+0.j]]
[[ 0.028205+0.041926j]
 [-0.067001+0.065267j]
 [-0.082401+0.0359j  ]
 [-0.144517-0.098328j]
 [-0.082401+0.0359j  ]
 [-0.091309-0.133586j]
 [-0.094568-0.132406j]
 [ 0.12022 -0.29567j ]
 [-0.067001+0.065267j]
 [-0.140676-0.100495j]
 [-0.091309-0.133586j]
 [ 0.124697-0.288969j]
 [-0.144517-0.098328j]
 [ 0.124697-0.288969j]
 [ 0.12022 -0.29567j ]
 [ 0.616243-0.111121j]]


In [66]:
x = [1,2,3]
x += np.array([2,3, 4]).tolist()
print(x)


[1, 2, 3, 2, 3, 4]


In [68]:
q.entropy_subsys(qe, )


TypeError: bipartite_spectral_fn() missing 2 required positional arguments: 'dims' and 'sysa'

In [224]:
qe_ = q.ket([1, 0])
qg_ = q.ket([0, 1])
qeee = q.kron(qe_, qe_, qe_)
qggg = q.kron(qg_, qg_, qg_)
q_ghz = q.normalize(qeee + qggg)
q_all = q.normalize(
    q.kron(qg_, qg_, qg_) +
    q.kron(qe_, qg_, qg_) +
    q.kron(qg_, qe_, qg_) +
    q.kron(qg_, qg_, qe_) +
    q.kron(qe_, qe_, qg_) +
    q.kron(qe_, qg_, qe_) +
    q.kron(qg_, qe_, qe_) +
    q.kron(qe_, qe_, qe_)    
)
print(qggg)
print(qeee)
print(q_ghz)
print(q_all)

[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [1.+0.j]]
[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[0.707107+0.j]
 [0.      +0.j]
 [0.      +0.j]
 [0.      +0.j]
 [0.      +0.j]
 [0.      +0.j]
 [0.      +0.j]
 [0.707107+0.j]]
[[0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]
 [0.353553+0.j]]


In [81]:
q.entropy_subsys(qeee, [2] * 3 , [0])

0.0

In [238]:
q.entropy_subsys(q_ghz, [2] * 3, [2])

1.0

In [190]:
q.entropy_subsys(q_all, [2] * 3, [2])

0.5500477595827573

In [198]:
qeee @ qeee.H

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]

In [199]:
qggg @ qggg.H

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]

In [201]:
q_ghz_rho = q_ghz @ q_ghz.H
q_ghz_rho

[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j]]

In [225]:
for i in [qggg, qeee, q_ghz, q_all]:
    print(q.entropy(i @ i.H))


0.0
0.0
0.0
1.1185094152656821e-14


In [239]:
test_state = q_ghz
test_density_matrix = test_state @ test_state.H
print(-np.trace(np.nan_to_num(test_density_matrix * np.log(test_density_matrix))))
print(-np.sum(test_state ** 2 * np.nan_to_num(np.log(test_state ** 2))))

C:\Users\harry\Anaconda3\envs\urop\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\harry\Anaconda3\envs\urop\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\harry\Anaconda3\envs\urop\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


(0.6931471805599454-0j)
(0.6931471805599454-0j)


In [227]:
np.e ** 2.079


7.996468446282769